# Deep Temporal Transformer for Financial Fraud Detection
**Author:** Prasad Kharat  
**GPU Optimized** - Works on any GPU (T4, V100, A100)

This notebook demonstrates the complete fraud detection pipeline.

## 1. Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Install dependencies
!pip install torch torchvision scikit-learn pandas matplotlib seaborn -q
print("✅ Dependencies installed!")

✅ Dependencies installed!


## 2. Upload Your Code

**Option A:** Upload the entire project folder as a zip file, then unzip it.

**Option B:** Clone from GitHub (if you've pushed it):
```python
!git clone https://github.com/Prime-21/Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection.git
%cd Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection
```

**Option C:** Use the file upload below:

In [ ]:
!rm -rf Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection

In [ ]:
!git clone https://github.com/Prime-21/Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection.git
%cd Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection

Cloning into 'Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 103 (delta 32), reused 65 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 95.45 KiB | 3.53 MiB/s, done.
Resolving deltas: 100% (32/32), done.
/content/Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection/Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection/Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection/Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection/Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection


In [ ]:
# Option C: Upload zip file
from google.colab import files
import zipfile

print("Upload your project zip file:")
uploaded = files.upload()

# Unzip
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"✅ Extracted {filename}")

# Navigate to directory
%cd Deep-Temporal-Transformer-For-High-Financial-Fraud-Detection

Upload your project zip file:


: 

## 3. Install the Package

In [ ]:
# Install the package in development mode
!pip install -e . -q
print("✅ Package installed!")

  Preparing metadata (setup.py) ... done
✅ Package installed!


## 4. Quick Demo - Basic Usage

In [ ]:
# Restart the Python kernel to reload the module
import sys
if 'deep_temporal_transformer' in sys.modules:
    del sys.modules['deep_temporal_transformer']

# Now import should work
from deep_temporal_transformer import (
    get_default_config, DataProcessor, ModelTrainer,
    set_random_seeds, get_device
)

set_random_seeds(42)
config = get_default_config()
device = get_device()

print(f"✅ Imports successful!")
print(f"🚀 Using device: {device}")
print(f"📊 Model config: d_model={config.model.d_model}, layers={config.model.num_layers}")

ModuleNotFoundError: No module named 'deep_temporal_transformer.data.utils'

In [ ]:
from deep_temporal_transformer import (
    get_default_config, DataProcessor, ModelTrainer,
    set_random_seeds, get_device
)

# Setup
set_random_seeds(42)
config = get_default_config()
device = get_device()

print(f"🚀 Using device: {device}")
print(f"📊 Model config: d_model={config.model.d_model}, layers={config.model.num_layers}")

ModuleNotFoundError: No module named 'deep_temporal_transformer.models.model'

In [ ]:
# Load and process data (using synthetic data for demo)
processor = DataProcessor(seq_len=8, random_state=42)
X_train, y_train, X_val, y_val, X_test, y_test = processor.process_data()

print(f"✅ Data loaded:")
print(f"  Train: {X_train.shape}")
print(f"  Val: {X_val.shape}")
print(f"  Test: {X_test.shape}")
print(f"  Fraud rate: {y_train.mean():.2%}")

In [ ]:
# ⚡ GPU-OPTIMIZED TRAINING for 15GB GPU
config.training.batch_size = 2048      # 🔥 8x larger (was 256)
config.training.epochs = 20            # More epochs
config.training.learning_rate = 5e-4   # Higher LR for larger batches

# 🧠 LARGER MODEL (uses 10-12 GB GPU)
config.model.d_model = 512             # Was 256
config.model.num_layers = 8            # Was 6
config.model.nhead = 16                # Was 8
config.model.dim_feedforward = 2048    # Was 1024
config.model.memory_slots = 2048       # Was 1024

trainer = ModelTrainer(config, device)
trainer.setup_model(input_dim=X_train.shape[-1])

print(f"🧠 Model: {sum(p.numel() for p in trainer.model.parameters()):,} parameters")
print(f"📊 Batch size: {config.training.batch_size}")
print(f"⚡ Expected GPU: 10-12 GB / 15 GB")
print("\n🎯 Starting training...")

history = trainer.train(X_train, y_train, X_val, y_val)

# Monitor GPU usage
import torch
gpu_used = torch.cuda.max_memory_allocated() / 1024**3
print(f"\n📊 GPU Used: {gpu_used:.2f} GB ({gpu_used/15*100:.1f}%)")


In [ ]:
# Evaluate on test set
results = trainer.evaluate_model(X_test, y_test)

print("\n" + "="*60)
print("📊 TEST SET RESULTS")
print("="*60)
print(f"F1 Score:       {results['f1']:.4f}")
print(f"AUC Score:      {results['auc']:.4f}")
print(f"Precision:      {results['precision']:.4f}")
print(f"Recall:         {results['recall']:.4f}")
print(f"Inference Time: {results['avg_inference_time']:.6f}s per transaction")
print("="*60)

## 5. Advanced Usage - GPU-Optimized Model

In [ ]:
from deep_temporal_transformer.models.advanced_transformer import DeepTemporalTransformerAdvanced
from deep_temporal_transformer.training.advanced_training import (
    detect_and_configure_gpu, FocalLossAdvanced
)

# Auto-detect and configure GPU
gpu_config = detect_and_configure_gpu()
device = gpu_config['device']

print("\n🚀 GPU Configuration:")
print(f"  GPU: {gpu_config['gpu_name']}")
print(f"  Memory: {gpu_config['gpu_memory_gb']:.1f} GB")
print(f"  Mixed Precision: {'BF16' if gpu_config['use_bf16'] else ('FP16' if gpu_config['use_fp16'] else 'FP32')}")
print(f"  Batch Size Multiplier: {gpu_config['batch_size_multiplier']:.1f}x")

In [ ]:
# Initialize advanced model with all innovations
model = DeepTemporalTransformerAdvanced(
    input_dim=X_train.shape[-1],
    d_model=256,
    num_heads=8,
    num_layers=6,
    num_experts=8,
    memory_slots=512,
    use_gradient_checkpointing=True  # Enable for memory efficiency
).to(device)

print(f"\n🧠 Advanced Model Initialized")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Features: Sparse Attention + MoE + Temporal Modules + External Memory")

In [ ]:
# Example: Predict with uncertainty estimation
import torch

# Convert test data to tensor
X_test_tensor = torch.FloatTensor(X_test[:100]).to(device)

# Get predictions with uncertainty
mean_probs, uncertainty = model.predict_with_uncertainty(X_test_tensor, n_samples=10)

print("\n🎯 Uncertainty-Aware Predictions (first 5):")
for i in range(5):
    print(f"  Sample {i+1}: Fraud Prob={mean_probs[i]:.4f}, Uncertainty={uncertainty[i]:.4f}")

## 6. Visualizations

In [ ]:
from deep_temporal_transformer.evaluation.explain import ModelExplainer
import matplotlib.pyplot as plt

# Create explainer
explainer = ModelExplainer(trainer.model, device)

# Plot training history
explainer.plot_training_history(history)
plt.show()

# Plot confusion matrix
explainer.plot_confusion_matrix(X_test, y_test)
plt.show()

print("✅ Visualizations complete!")

## 7. Save Model (Optional)

In [ ]:
# Save to Google Drive (optional)
from google.colab import drive
drive.mount('/content/drive')

# Save model
save_path = '/content/drive/MyDrive/fraud_detection_model.pt'
trainer.save_model(save_path)
print(f"✅ Model saved to: {save_path}")

## 🎉 Complete!

You've successfully:
- ✅ Set up the environment
- ✅ Trained the Deep Temporal Transformer
- ✅ Evaluated performance
- ✅ Explored advanced features
- ✅ Generated visualizations

**Next Steps:**
- Use your own dataset instead of synthetic data
- Experiment with hyperparameters
- Try the advanced model with uncertainty estimation
- Generate thesis plots and results